(10 points) (Exercise 7.9) For the prostate data of Chapter 3, carry out a bestsubset linear regression analysis,
as in Table 3.3 (third column from the left). Compute the AIC, BIC, five- and tenfold cross-validation, and bootstrap .632 estimates of prediction error.

In [56]:
import pandas as pd
import numpy as np
import statsmodels
import time
import itertools
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture # AIC BIC
from sklearn.linear_model import LinearRegression , LogisticRegression
from sklearn.linear_model import LassoLarsIC
from sklearn.model_selection import KFold  # or just kfold with n 
from sklearn.neighbors import KNeighborsClassifier
from statsmodels.tsa.arima_model import ARMAResults 


import statsmodels.api as sm
import statsmodels.formula.api as smf

from mlxtend.feature_selection import ExhaustiveFeatureSelector as BSS # includes forward and backwawrds selection

In [3]:
df = pd.read_csv('./PROSTATE_DATA/prostate.csv', delimiter="\t") 

In [4]:
# INPUT PROSTATE DATA
prostate_data = df.set_index('Unnamed: 0') 
del prostate_data.index.name
print(prostate_data.head())

     lcavol   lweight  age      lbph  svi       lcp  gleason  pgg45      lpsa  \
1 -0.579818  2.769459   50 -1.386294    0 -1.386294        6      0 -0.430783   
2 -0.994252  3.319626   58 -1.386294    0 -1.386294        6      0 -0.162519   
3 -0.510826  2.691243   74 -1.386294    0 -1.386294        7     20 -0.162519   
4 -1.203973  3.282789   58 -1.386294    0 -1.386294        6      0 -0.162519   
5  0.751416  3.432373   62 -1.386294    0 -1.386294        6      0  0.371564   

  train  
1     T  
2     T  
3     T  
4     T  
5     T  


In [65]:
prostate_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 1 to 97
Data columns (total 10 columns):
lcavol     97 non-null float64
lweight    97 non-null float64
age        97 non-null int64
lbph       97 non-null float64
svi        97 non-null int64
lcp        97 non-null float64
gleason    97 non-null int64
pgg45      97 non-null int64
lpsa       97 non-null float64
train      97 non-null object
dtypes: float64(5), int64(4), object(1)
memory usage: 8.3+ KB


In [5]:
prostate_data_target = prostate_data.train
prostate_data_predictors = prostate_data.drop('train', axis = 1)  
# print(prostate_data_target.tolist())

In [6]:
# print(len(prostate_data_predictors))
true_false_list = []
prostate_list = prostate_data_target.tolist()
for i in range(0 , len(prostate_data_target)): 
    if prostate_list[i] == "T":
        true_false_list.append(True)
    else :
        true_false_list.append(False)
true_false_list = pd.DataFrame({"Target" : true_false_list})
prostate_data_target = true_false_list
print(prostate_data_target)

    Target
0     True
1     True
2     True
3     True
4     True
..     ...
92    True
93    True
94   False
95    True
96   False

[97 rows x 1 columns]


In [7]:
x_train,x_test,y_train,y_test=train_test_split(prostate_data_predictors,prostate_data_target,test_size=0.2)

In [12]:
type(prostate_data_target["Target"])

pandas.core.series.Series

In [47]:
prostate_data_predictors.shape

(97, 9)

In [30]:
lm = LinearRegression()
# lm = LogisticRegression()
# lm = LassoLarsIC()
# knn = KNeighborsClassifier(n_neighbors=3)

bss = BSS(lm,   
           scoring='neg_mean_squared_error',
           print_progress=True,
           cv=5) 
# bss = BSS(lm, 
#            min_features=1,
#            max_features=4,
#            scoring='accuracy',
#            print_progress=True,
#            cv=5) 


bss = bss.fit(prostate_data_predictors, prostate_data_target) 

Features: 9/9

In [31]:

print(bss.best_score_ * (-1))

0.21689136719284496


In [32]:

print('Best subset:', bss.best_idx_)

Best subset: (2,)


In [40]:
best_score = bss.best_score_ * (-1)
print('Best accuracy score: %.2f' % best_score)
print('Best subset (indices):', bss.best_idx_)
print('Best subset (corresponding names):', bss.best_feature_names_)

Best accuracy score: 0.22
Best subset (indices): (2,)
Best subset (corresponding names): ('age',)


In [44]:
models_best = pd.DataFrame(columns=["RSS", "model"])

In [49]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(prostate_data_predictors,prostate_data_predictors[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(prostate_data_predictors[list(feature_set)]) - prostate_data_predictors) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [82]:
def getBest(k):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(prostate_data_predictors.columns, k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    print(models.loc[0])
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [83]:
# Could take quite awhile to complete...

models_best = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
for i in range(1,len(prostate_data_predictors.columns)):
    models_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

model    <statsmodels.regression.linear_model.Regressio...
RSS      0          0.0
1          0.0
2          0.0
3...
Name: 0, dtype: object


/home/mavericku/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  from ipykernel import kernelapp as app


TypeError: reduction operation 'argmin' not allowed for this dtype